In [65]:
# Import Pandas lib
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime

# Import sklearn lib
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Import keras lib
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Other support lib
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np

C:\Users\Kraing\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [66]:
# create a differenced series -> CURRENT VALUE MINUS LAST
def difference(dataset, interval=1):
    diff = list()
    for row in range(interval, len(dataset)):
        # dx
        dx = dataset[row][1] - dataset[row - interval][1]
        dy = dataset[row][2] - dataset[row - interval][2]
        dv = dataset[row][3] - dataset[row - interval][3]
        diff.append([dx, dy, dv])
    return diff

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled


# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    print(X.shape)
    print(value.shape)
    new_row = [x for x in X] #+ [value]
    
    new_row.append(value[0])
    new_row.append(value[1])
    new_row.append(value[2])
    
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]


# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    
    # X =  feature -> prime 3 colonne
    # y = label -> ultime 3 colonne
    X, y = train[:, 0:3], train[:, 3:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    print(y.shape)
    
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
        
    return model


# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    #yhat is 2-D array
    return yhat[0, :]

In [67]:
# Load Time series-Data
raw_series = read_csv('Traces/75533_T.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)

# Drop ID column
raw_series = raw_series.drop(['id'], axis=1)

# Drop header and row-index
raw_values = raw_series.values
diff_values = difference(raw_values, 1)

#print(diff_values)
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)

# Ogni riga è composta da Current(dx, dy, dv) - Future(dx, dy, dv)
supervised_values = supervised.values

# split data into train and test-sets  -> traininset = 80%
m_train = int(len(supervised_values) * 0.8)
train, test = supervised_values[0:m_train], supervised_values[m_train:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# fit the model
lstm_model = fit_lstm(train_scaled, batch_size=1, nb_epoch=1500, neurons=1)



(117, 3)


In [68]:
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0:3].reshape(len(train_scaled), 1, 3)
tmp = lstm_model.predict(train_reshaped, batch_size=1)
print(tmp.shape)

(117, 3)


In [69]:
# walk-forward validation on the test data
predictions = list()

for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:3], test_scaled[i, 3:-1]
    
    yhat = forecast_lstm(lstm_model, 1, X)

    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    
    print(yhat)
    '''
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    
    # store forecast
    predictions.append(yhat)
    expected = raw_values[len(train) + i + 1]
    print(f'Time-Step={i}, Predicted={yhat}, Expected={expected}')
    
# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)
'''

(3,)
(3,)
0.019884891882537933
(3,)
(3,)
-0.013968263193965817
(3,)
(3,)
-0.10029634051024999
(3,)
(3,)
-0.07015523843467321
(3,)
(3,)
-0.2134856796264657
(3,)
(3,)
-0.23225863151252354
(3,)
(3,)
-0.2518673919141301
(3,)
(3,)
-0.2585665193945178
(3,)
(3,)
-0.2253075779974469
(3,)
(3,)
-0.33777882605791176
(3,)
(3,)
-0.23896838121116246
(3,)
(3,)
-0.2741720131039628
(3,)
(3,)
-0.26854495361447417
(3,)
(3,)
-0.25270223036408507
(3,)
(3,)
-0.270581024289132
(3,)
(3,)
-0.25536942571401683
(3,)
(3,)
-0.2600974769145259
(3,)
(3,)
-0.24563806720078077
(3,)
(3,)
-0.26906199857592666
(3,)
(3,)
-0.2858654747903355
(3,)
(3,)
-0.4918769468367108
(3,)
(3,)
-0.20882048867642963
(3,)
(3,)
-0.25750318199396216
(3,)
(3,)
-0.2658652444183835
(3,)
(3,)
-0.29328545801341616
(3,)
(3,)
-0.3013109561800965
(3,)
(3,)
-0.30259959131479347
(3,)
(3,)
-0.2944618565589198
(3,)
(3,)
-0.31247194886207663
(3,)
(3,)
-0.3025804810225972


In [75]:
# Plot
x = raw_series['x'].values
y = raw_series['y'].values

print(x)
'''
plt.figure()
plt.show(block=False)
for i in range(len(raw_series)):
    plt.scatter(x[i], y[i], s=10)
    plt.draw()
    plt.pause(0.5)
'''
#plt.scatter(raw_series['x'], raw_series['y'], s=10)


[13058.98572223 13061.08920073 13063.16916207 13065.1393767
 13067.20104693 13068.88644274 13069.68602587 13076.78296356
 13082.79681725 13089.6958664  13096.27551513 13103.08330702
 13110.13749352 13116.80839951 13124.07247772 13130.72513226
 13137.32303244 13144.47182343 13151.46642429 13158.03185251
 13165.1451607  13192.84665639 13196.20247446 13200.42222192
 13205.49246194 13210.79322976 13215.86099785 13221.27678397
 13226.32500048 13231.71732471 13237.06200625 13241.63619056
 13244.62806047 13246.71687253 13231.84010927 13223.05446628
 13212.54616877 13200.65205213 13187.10073263 13156.39411603
 13134.43847354 13122.44284676 13110.00295553 13098.42526759
 13086.59244822 13074.70684304 13063.29470599 13051.74250598
 13025.67680695 13013.58578545 13001.59529399 12979.51920874
 12968.59445161 12957.51374378 12946.51511527 12936.57530995
 12924.72531721 12921.62834779 12919.9091404  12907.80047572
 12901.70718881 12893.99243899 12884.8439746  12874.38127186
 12863.07370161 12851.561

'\nplt.figure()\nplt.show(block=False)\nfor i in range(len(raw_series)):\n    plt.scatter(x[i], y[i], s=10)\n    plt.draw()\n    plt.pause(0.5)\n'